# Languages, Parsing, and Abstract Syntax Trees

These notes cover several background concepts, definitions, and constructs that will allow us to describe and reason about programming languages.

## Dependencies

In [73]:
# Presentation dependencies.
%matplotlib inline
%config InlineBackend.figure_format='retina'
import matplotlib as mp
import matplotlib.pyplot as plt
from importlib import reload
from IPython.display import Image
from IPython.display import display_html
from IPython.display import display
from IPython.display import Math
from IPython.display import Latex
from IPython.display import HTML

# Content dependencies (also reproduced inline).
from random import randint
from itertools import permutations
from functools import reduce
from tqdm import tqdm

## Mathematical Background and Prerequisites

**Basic logic.** It is expected that the reader is familiar with the basic concepts of mathematical logic, including formulas, predicates, relational operators, and terms.

**Basic set theory.** It is expected that the reader is familiar with the notion of a set (both finite and infinite), set size, and set comprehension notation (e.g., $\{1,2,3,4\}$ is a set and $\{x \ | \ x \in \mathbb{Z}, x > 0 \} = \{1, 2, 3, ...\}$). Is it also expected that the reader is familiar with the membership and subset relations between elements and sets, (e.g., $1 \in \{1,2,3\}$ and $\{2,3\} \subset \{1,2,3\}$). 


## Defining Programming Languages

In order to define and reason about programming languages, and in order to write automated tools and algorithms that can operate on programs written using programming languagues, we must be able to define formally (i.e., mathematically) what is a programming language and what is a program.

### Sets of Character Strings

In computer science, one common way to mathematically model a formal language is to introduce a finite set of symbols (an *alphabet*). A *language* is then any subset of the set of all strings consisting of symbols from that alphabet.

<div style="background-color:#F6F4D8; padding:5px 8px 5px 8px;">
<b>Definition.</b> An <i>alphabet</i> is a finite set $A$. We will call elements $a \in A$ of the set <i>characters</i>. The typical alphabet we will use in this course is the set of 128 ASCII characters. However, any finite set can be an alphabet.
</div>

<div style="background-color:#F6F4D8; padding:5px 8px 5px 8px;">
<b>Definition.</b> Given an alphabet $A$, a <i>character string</i> or <i>string</i> is any ordered finite sequence of characters from that alphabet. We will denote the empty string (containing no characters) using the symbol $\varepsilon$, and we will denote the length of a string $s$ using the notation $|s|$.
</div>

<div style="background-color:#F6F4D8; padding:5px 8px 5px 8px;">
<b>Definition.</b> Given an alphabet $A$, let $U$ be the set of all finite strings that can be built using characters from $A$ (including the empty string $\varepsilon$). In other words, we can say:
<span style="height:25px; display:block;"></span>
$$U \ = \ \{s \ | \ s \ \mathrm{is \ a \ finite \ string \ of \ characters \ from \ alphabet} \ A\}$$
<span style="height:25px; display:block;"></span>
Any subset $L \subset U$ is a <i>language</i>. That is, any set of character strings (whether the set is finite or infinite) is a language.
</div>

### Sets of Token Sequences

Unlike human languagues, programming languages usually have a relatively small collection of symbol strings (e.g., commands or instructions) that are used to construct programs. Thus, we can adjust the definition of what constitutes a language to account for this.

<div style="background-color:#F6F4D8; padding:5px 8px 5px 8px;">
<b>Definition.</b> Given an alphabet $A$, a <i>token</i> is a finite, non-empty (usually short) string of characters from that alphabet.
</div>

<div style="background-color:#F6F4D8; padding:5px 8px 5px 8px;">
<b>Definition.</b> Given a set of tokens $T$, let $U$ be the set of all finite sequences that can be built using tokens from $T$ (including the empty sequence). In other words, we can say:
<span style="height:25px; display:block;"></span>
$$U \ = \ \{s \ | \ s \ \mathrm{is \ a \ finite \ sequence \ of \ tokens \ from} \ T\}$$
<span style="height:25px; display:block;"></span>
Any subset $L \subset U$ is a <i>language</i>. That is, any set of token sequences (whether the set is finite or infinite) is a language.
</div>

<div style="background-color:#E1F8E1; padding:5px 8px 5px 8px;">
<b>Example.</b> Consider the following set of tokens:
<span style="height:25px; display:block;"></span>
$$T = \{\textbf{true}, \textbf{false}, \textbf{or}, \textbf{and}, \textbf{not}, \textbf{(}, \textbf{)}, \textbf{,}\}$$
<span style="height:25px; display:block;"></span>
The set of token sequences that represent valid boolean formulas is a language:
<span style="height:25px; display:block;"></span>
$$L = \{\ \textbf{true} \ , \ \textbf{false} \ , \ \textbf{not ( false )} \ , \ \textbf{and ( true , false )} \ , \ldots \ \}$$
<span style="height:25px; display:block;"></span>
Note that $L$ is an infinitely large set.
</div>

If a language is just a subset of the set of all possible token sequences, how do we mathematically specify interesting subsets?

### Formal Tools for Defining Languages

A variety of formal tools and techniques exist within computer science and mathematics to formally define, identify, or delineate interesting subsets of the set of all strings. These range in their expressive power from [regular expressions](https://en.wikipedia.org/wiki/Regular_expression) (or, equivalently, [finite state automata](https://en.wikipedia.org/wiki/Finite-state_machine)) to [Turing machines](https://en.wikipedia.org/wiki/Turing_machine). A full review of these techniques is beyond the scope of this course.

<div style="background-color:#F6F4D8; padding:5px 8px 5px 8px;">
<b>Definition.</b> Given a token set $T$ and a language $L$ consisting of finite sequences of tokens from $T$, the <i>syntax</i> of $L$ is a collection of formal rules defining <i>exactly</i> which token sequences are in $L$. These rules are sometimes also called <i>syntactic rules</i>, a <i>formal grammar</i>, or simply a <i>grammar</i>.
</div>

In these notes, we will focus on a common formal technique for defining languages that has been adopted by the creators of programming languages: [context-free grammars](https://en.wikipedia.org/wiki/Context-free_grammar) (CFGs). While CFGs are not the most powerful formal tool for defining languages, they are sufficient to define the kinds of recursive language patterns found in programming languages that are used in practice. The most common representation for such grammars is [Backus-Naur Form](https://en.wikipedia.org/wiki/Backus%E2%80%93Naur_form), abbreviated henceforward as BNF.

The production rules in a grammar's BNF representation can be viewed both as a way to construct an element (i.e., a token sequence that is a program) in the language, or as a way to break down (i.e., *parse*) a token sequence piece by piece until nothing is left.

<div style="background-color:#F6F4D8; padding:5px 8px 5px 8px;">
<b>Definition.</b> Given an alphabet, token set, and grammar definition (e.g., represented using BNF notation), we define the <i>concrete syntax</i> to be the set of all character strings (or, equivalently, token sequences) that conform to the grammar definition. We call a particular character string (or token sequence) that conforms to the grammar definition a <i>concrete syntax instance</i>.
</div>

<div style="background-color:#E1F8E1; padding:5px 8px 5px 8px;">
<b>Example.</b> Let $T$ be a set of tokens:
<span style="height:25px; display:block;"></span>
$$T = \{\textbf{true}\}$$
<span style="height:25px; display:block;"></span>
The following is a grammar for a very simple programming language that contains only one possible token sequence consisting of a single token $\textbf{true}$:
<span style="height:25px; display:block;"></span>
$$\textit{program} ::= \textbf{true}$$
<span style="height:25px; display:block;"></span>
In this case, the language is finite and small, so we can write it down as a set:
<span style="height:25px; display:block;"></span>
$$L = \{ \ \textbf{true} \ \}$$
<span style="height:25px; display:block;"></span>
</div>

<div style="background-color:#E1F8E1; padding:5px 8px 5px 8px;">
<b>Example.</b> We can extend the language in the above example by adding another token:
<span style="height:25px; display:block;"></span>
$$T = \{\textbf{true}, \textbf{false}\}$$
<span style="height:25px; display:block;"></span>
The following BNF grammar definition now contains two choices (each choice is a sequence consisting of a single terminal):
<span style="height:25px; display:block;"></span>
\begin{eqnarray*}
\textit{program} & ::= & \textbf{true} \\
                 &  |  & \textbf{false} \\
\end{eqnarray*}
<span style="height:25px; display:block;"></span>
The programming language that satisfies the above grammar contains two token sequences (each consisting of a single token):
<span style="height:25px; display:block;"></span>
$$L = \{ \ \textbf{true} \ , \ \textbf{false} \ \}$$
<span style="height:25px; display:block;"></span>
</div>

<div style="background-color:#E1F8E1; padding:5px 8px 5px 8px;">
<b>Example.</b> Extending the above language definition further, we can construct a language for Boolean formulas. The set of tokens $T$ would be defined as follows: 
<span style="height:25px; display:block;"></span>
$$T = \{\textbf{true}, \textbf{false}, \textbf{or}, \textbf{and}, \textbf{not}, \textbf{(}, \textbf{)}, \textbf{,}\}$$
<span style="height:25px; display:block;"></span>
The following BNF grammar definition now contains five choices (each choice is a sequence consisting of non-terminals and/or terminals):
<span style="height:25px; display:block;"></span>
\begin{eqnarray*}
\textit{program} & ::= & \textbf{true} \\
                 &  |  & \textbf{false} \\
                 &  |  & \textbf{and} \ \textbf{(} \ \textit{program} \ \textbf{,} \ \textit{program} \ \textbf{)}\\
                 &  |  & \textbf{or} \ \textbf{(} \ \textit{program} \ \textbf{,} \ \textit{program} \ \textbf{)}\\
                 &  |  & \textbf{not} \ \textbf{(} \ \textit{program} \ \textbf{)}\\
\end{eqnarray*}
<span style="height:25px; display:block;"></span>
This programming language now contains infinitely many finite token sequences:
<span style="height:25px; display:block;"></span>
$$L = \{\ \textbf{true} \ , \ \textbf{false} \ , \ \textbf{not ( false )} \ , \ \textbf{and ( true , false )} \ , \ldots \ \}$$
<span style="height:25px; display:block;"></span>
</div>

<div style="background-color:#E1F8E1; padding:5px 8px 5px 8px;">
<a name="85feea29-7b81-4b58-bf08-bf8b0b2195e9"></a>
<b>Example [<a href="#85feea29-7b81-4b58-bf08-bf8b0b2195e9">link</a>].</b> We can extend the language by adding a production rule for statements, and then by allowing a program to be a sequence of one or more statements (that themselves may contain formulas). The set of tokens is defined below:
<span style="height:25px; display:block;"></span>
$$T = \{\textbf{true}, \textbf{false}, \textbf{or}, \textbf{and}, \textbf{not}, \textbf{(}, \textbf{)}, \textbf{,}, \textbf{print}, \textbf{skip}, \textbf{;}\}$$
<span style="height:25px; display:block;"></span>
Note that in such a scenario, we would distinguish one of the production as the one representing token sequences in our language; in this example, that production rule is the one for $\textit{program}$. The following is the grammar definition for this language:
<span style="height:25px; display:block;"></span>
\begin{eqnarray*}
\textit{program} & ::= & \textit{statement} \\
                 &  |  & \textit{statement} \ \textit{program} \\
 & & \\
\textit{statement} & ::= & \textbf{print} \ \textit{formula} \ \textbf{;} \\
                 &  |  & \textbf{skip} \ \textbf{;} \\
 & & \\
\textit{formula} & ::= & \textbf{true} \\
                 &  |  & \textbf{false} \\
                 &  |  & \textbf{and} \ \textbf{(} \ \textit{formula} \ \textbf{,} \ \textit{formula} \ \textbf{)} \\
                 &  |  & \textbf{or} \ \textbf{(} \ \textit{formula} \ \textbf{,} \ \textit{formula} \ \textbf{)}\\
                 &  |  & \textbf{not} \ \textbf{(} \ \textit{formula} \ \textbf{)}\\
\end{eqnarray*}
<span style="height:25px; display:block;"></span>
</div>

## Parsing, Abstract Syntax, and Abstract Syntax Trees

Given a programming language definition, we want to have the ability to operate on programs written in that language using an automated process. To do so, we need to convert the character string representations of programs in that programming language (i.e., the concrete syntax representation) into instances of a data structure; each data structure instance would then be a representation of a program in that language as data on which we can operate.

### Abstract Syntax

<div style="background-color:#F6F4D8; padding:5px 8px 5px 8px;">
<b>Definition.</b> For a particular programming language definition, we define the <i>abstract syntax</i> to be the set of all data structure instances that correspond to a character string (or token sequence) that conforms to the grammar definition for that language. An instance of the abstract syntax data structure is sometimes called a <i>parse tree</i>, an <i>abstract syntax tree</i>, or <i>AST</i>.
</div>

<div style="background-color:#E1F8E1; padding:5px 8px 5px 8px;">
<b>Example.</b> Consider again the language that conforms to the following grammar:
<span style="height:25px; display:block;"></span>
\begin{eqnarray*}
\textit{formula} & ::= & \textbf{true} \ \ | \ \ \textbf{false} \ \ | \ \ \textbf{not} \ \textbf{(} \ \textit{formula} \ \textbf{)} \\
                 &  |  & \textbf{and} \ \textbf{(} \ \textit{formula} \ \textbf{,} \ \textit{formula} \ \textbf{)} \ \ | \ \ \textbf{or} \ \textbf{(} \ \textit{formula} \ \textbf{,} \ \textit{formula} \ \textbf{)}\\
\end{eqnarray*}
<span style="height:25px; display:block;"></span>
The following is the character string of one possible program in the language. This character string is an instance of the concrete syntax of the language.

<pre>
and (or (and (true, false), not(false)), or (true, false))
</pre>

The above character string might be converted into a structured representation of the program within some host language (i.e., the programming language being used to operate on these programs: checking for errors, interpreting, or compiling the program). Below, we present one possible Python representation of the instance of the abstract syntax (i.e., the parse tree) corresponding to the concrete syntax instance above. This representation uses nested Python dictionaries to represent the parse tree, with strings being used to represent node labels and leaves.

<pre>
{ "And": [
    { "Or": [
        { "And": ["True","False"]}, 
        { "Not": ["False"]}
      ]
    }, 
    { "Or": ["True","False"]}
  ]
}
</pre>
</div>

<div style="background-color:#E1F8E1; padding:5px 8px 5px 8px;">
<b>Example.</b> The following is an example of a formal abstract syntax definition:
<span style="height:25px; display:block;"></span>
\begin{eqnarray*}
\textit{formula} & ::= & \textbf{true} \ \ | \ \ \textbf{false} \ \ | \ \ \textbf{not} \ \textit{formula} \\
                 &  |  & \textit{formula} \ \textbf{and} \ \textit{formula} \ \ | \ \ \textit{formula} \ \textbf{or} \ \textit{formula}\\
\end{eqnarray*}
<span style="height:25px; display:block;"></span>
Notice the omission of the parentheses. Also, there is no need to be concerned with the fixity (i.e., infix vs. prefix) of binary operators, since this definition is not being used to implement a parsing algorithm. All that is important is that there is enough information distinguish the different types of abstract syntax tree nodes with distinct labels.
</div>

### Parsing Python using Python

The [`ast`](https://docs.python.org/3/library/ast.html) module in Python provides an abstract syntax data structure, as well as associated functionalities (such as parsing a concrete syntax instance into an AST and performing transformations on an AST).

In [1]:
import ast

# Programmatically build an AST for the
# Python concrete syntax expression `-5`.
node =\
  ast.UnaryOp(
    ast.USub(),
    ast.Num(5, lineno=0, col_offset=0),
    lineno=0,
    col_offset=0
  )

ast.dump(node)

'UnaryOp(op=USub(), operand=Num(n=5))'

We can use the [`inspect`](https://docs.python.org/3/library/inspect.html) module to retrieve the concrete syntax of the body of a Python function definition.

In [2]:
def f(x):
    return x + 2

import inspect
inspect.getsource(f)

'def f(x):\n    return x + 2\n'

We can then parse the concrete syntax using [`ast.parse()`](https://docs.python.org/3/library/ast.html#ast.parse). We demonstate step-by-step how this object can be inspected and traversed using the `ast` API.

In [76]:
t = ast.parse(inspect.getsource(f))
ast.dump(t)

"Module(body=[FunctionDef(name='f', args=arguments(args=[arg(arg='x', annotation=None)], vararg=None, kwonlyargs=[], kw_defaults=[], kwarg=None, defaults=[]), body=[Return(value=BinOp(left=Name(id='x', ctx=Load()), op=Add(), right=Num(n=2)))], decorator_list=[], returns=None)])"

In [77]:
# Get the body of the function.
print(t._fields)
print(t.body)
print(type(t.body[0]) is ast.FunctionDef)
func_body = t.body[0]

# Get the statement in the body of the function.
print(func_body._fields)
print(func_body.body)
print(type(func_body.body[0]) is ast.Return)
stmt_return = func_body.body[0]

# Get the expression inside the return statement.
print(stmt_return._fields)
print(stmt_return.value)
print(type(stmt_return.value) is ast.BinOp)
expr_add = stmt_return.value

# Look at the expression operator and its arguments.
print(expr_add.left)
print(expr_add.left.id)
print(expr_add.op)
print(expr_add.right)
print(expr_add.right.n)

('body',)
True
('name', 'args', 'body', 'decorator_list', 'returns')
True
('value',)
True
x
2


The `ast` API allows us to programmatically write our own functions for operating on Python ASTs.

In [16]:
def f(x, y):
    a = b
    u = 1 + 2
    z = x + y
    return z

def get_variables(a):
    if type(a) is ast.Module:
        return [v for s in a.body for v in get_variables(s)]
    
    elif type(a) is ast.FunctionDef:
        vs = [v for s in a.body for v in get_variables(s)]
        return [a.name] + vs

    elif type(a) is ast.Assign:
        vs = [v for s in a.targets for v in get_variables(s)]
        return vs + get_variables(a.value)
    
    elif type(a) is ast.Return:
        return get_variables(a.value)

    elif type(a) is ast.Name:
        return [a.id]

    elif type(a) is ast.BinOp:
        return get_variables(a.left) + get_variables(a.right)
    
    else:
        print(type(a))
        return []

a = ast.parse(inspect.getsource(f))
get_variables(a)


<class '_ast.Num'>
<class '_ast.Num'>


['f', 'a', 'b', 'u', 'z', 'x', 'y', 'z']

We can use the `ast.walk()` function to traverse a tree if we are not worried about its internal structure but wish to operate on its nodes.

In [17]:
vs = []
for node in ast.walk(a):
    if isinstance(node, ast.FunctionDef):
        vs.append(node.name)
    if isinstance(node, ast.Name):
        vs.append(node.id)
vs

['f', 'a', 'b', 'u', 'z', 'z', 'x', 'y']

We can use the `ast.NodeVisitor` and `ast.NodeTransformer` base classes to create our own algorithms that traverse or modify Python ASTs.

In [19]:
def f(x, y):
    a = b
    u = 1 + 2
    z = x + y
    return z

class VariableLister(ast.NodeVisitor):

    def visit_Name(self, node):
        print(node.id)
        self.generic_visit(node)
        
    def visit_FunctionDef(self, node):
        print(node.name)
        self.generic_visit(node) # Needed since we are overriding method.

a = ast.parse(inspect.getsource(f))
VariableLister().visit(a)

f
a
b
u
z
x
y
z


In [22]:
def f():
    return x + y

class VariableChanger(ast.NodeTransformer):

    def visit_Name(self, node):
        a =\
          ast.copy_location(
            ast.Subscript(
              value=ast.Name(id='data', ctx=ast.Load()),
              slice=ast.Index(value=ast.Str(s=node.id)),
              ctx=node.ctx
            ),
            node
          )
        return a

f_new_ast = VariableChanger().visit(ast.parse(inspect.getsource(f)))
ast.dump(f_new_ast)

"Module(body=[FunctionDef(name='f', args=arguments(args=[], vararg=None, kwonlyargs=[], kw_defaults=[], kwarg=None, defaults=[]), body=[Return(value=BinOp(left=Subscript(value=Name(id='data', ctx=Load()), slice=Index(value=Str(s='x')), ctx=Load()), op=Add(), right=Subscript(value=Name(id='data', ctx=Load()), slice=Index(value=Str(s='y')), ctx=Load())))], decorator_list=[], returns=None)])"

We can use the [`astor`](https://astor.readthedocs.io) module to convert an AST back into concrete syntax.

In [23]:
import astor
print(astor.to_source(f_new_ast))

def f():
    return data['x'] + data['y']



This is the end of the document.